In [33]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable

In [34]:
spark = (
    SparkSession
        .builder
        .appName("generate_symlink_manifest")
        .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.driver.extraClassPath","/usr/local/spark/jars/*") 
        .config("spark.sql.legacy.parquet.int96RebaseModeInWrite","CORRECTED") 
        .config("spark.sql.legacy.parquet.datetimeRebaseModeInWrite","CORRECTED") 
        .config("spark.sql.debug.maxToStringFields","1000")
        .getOrCreate()
)
# set log level to info
# [INFO] or [WARN] for more detailed logging info
spark.sparkContext.setLogLevel("WARN")

In [35]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

In [4]:
silver_crm_clientes = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/db_clientes/dbo/crm_clientes/'
silver_crm_clientes_restrito = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/db_clientes/dbo/crm_clientes_restrito/'
silver_hr_omni_pedido = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido/'
silver_hr_omni_pedido_restrito = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido_restrito/'

silver_hr_omni_pedido_item = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido_item/'
silver_loja_venda = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/loja_venda/'
silver_loja_venda_restrito = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/loja_venda_restrito/'
silver_produtos = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/ecommerce/bnsvher602/linx_db/dbo/produtos/'

In [5]:
def generete_symlink_manifest(spark:SparkSession, bucket:str) -> None:
  """
  Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
  """
  delta_table = DeltaTable.forPath(spark, bucket)
  delta_table.generate("symlink_format_manifest")

In [7]:
generete_symlink_manifest(spark, silver_hr_omni_pedido)

In [8]:
generete_symlink_manifest(spark, silver_hr_omni_pedido_item)

In [9]:
generete_symlink_manifest(spark, silver_hr_omni_pedido_restrito)

In [10]:
generete_symlink_manifest(spark, silver_produtos)

In [16]:
generete_symlink_manifest(spark, silver_crm_clientes)

In [22]:
generete_symlink_manifest(spark, silver_crm_clientes_restrito)

In [36]:
PEDIDO = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/BATCH/DB_MOBILE/PEDIDO/'
PEDIDO_ENTREGA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/BATCH/DB_MOBILE/PEDIDO_ENTREGA/'
PEDIDO_ITEM = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/BATCH/DB_MOBILE/PEDIDO_ITEM/'
HR_PRODUTOS_NEGOCIO = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/BATCH/LINX_DB/HR_PRODUTOS_NEGOCIO/'
VENDA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/BATCH/DB_MOBILE/VENDA/'
table_list = [VENDA]

In [37]:
for table in table_list:
    df = (
        spark
        .read
        .format("parquet")
        .load(table)
    )
    
    (
        df
        .coalesce(1)
        .write
        .mode("overwrite")
        .format("delta")
        .option("checkpointLocation", table + 'DELTA/' + "checkpoint")
        .save(table + 'DELTA/')
    )
    
    generete_symlink_manifest(spark, table + 'DELTA/')